# Hot Readme Generator (WIP)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://languageicon.org/language-icon.png"> 
</p>

</div>

## Description:

This app automates the generation of structured README files by creating a table of contents, integrating badges, and providing customizable configurations for headers, sections, and taglines. It uses a large language model (LLM) to tailor the content to specific project needs and ensures optimal formatting and professionalism.




## Step 1: Setup and Installation

This step installs required dependencies using install_requirements() and loads essential environment variables via setup_env(). It ensures all configurations, like OPENAI_API_KEY, are properly set up for the project.

In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Understanding the EasyLLM Class and Its Methods

This EasyLLM class is a Python wrapper around OpenAI's GPT API that simplifies interactions with the model. It includes methods for generating text, handling responses as objects, managing model selection, and estimating token usage.

By using EasyLLM, you can easily interact with OpenAI's GPT models without needing to manually handle API requests, set up system prompts, or process model responses.


In [ ]:
import os
import openai
from pydantic import BaseModel
import traceback
from typing import Union
import json
import re

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"
DEFAULT_SYSTEM_PROMPT = "You are an intelligent AI assistant. The user will give you a prompt, respond appropriately."
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 1024


class EasyLLM:
    """
    A simple abstraction for the OpenAI API. It provides easy-to-use methods to generate text and objects using the OpenAI API.
    A demonstration for the "How to build an Abstaction with Open AI API" blog post.
    Author: Aditya Patange (AdiPat)
    """

    def __init__(
        self,
        api_key=os.getenv("OPENAI_API_KEY"),
        model=DEFAULT_OPENAI_MODEL,
        verbose=True,
        debug=True,
    ):
        self.verbose = verbose
        self.debug = debug

        if self.verbose:
            print("EasyLLM: Powering up! 🚀")

        self.api_key = api_key
        self.openai = openai.OpenAI(api_key=api_key)
        self.model = model

        if self.verbose:
            print(f"EasyLLM: Model set to {model}.")
            print("EasyLLM: Ready for some Generative AI action! ⚡️")

    def generate_text(
        self,
        prompt: str,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[str, None]:
        """Generates text using the OpenAI API."""
        try:
            if self.verbose or self.debug:
                print(f"Generating text for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_tokens=max_tokens,
            )

            response = response.choices[0].message.content

            if self.verbose or self.debug:
                print("Text generated successfully. 🎉")

            if self.debug:
                response = json.dumps(response)
                print(f"EasyLLM Response: {response}")
            return response
        except Exception as e:
            print(f"Failed to generate text. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def generate_object(
        self,
        prompt: str,
        response_model: BaseModel,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[BaseModel, None]:
        """Generates an object using the OpenAI API and given response model."""
        try:
            if self.verbose or self.debug:
                print(f"Generating object for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")

            response = self.openai.beta.chat.completions.parse(
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                response_format=response_model,
                model=self.model,
                temperature=temperature,
                max_tokens=max_tokens,
            )

            if self.verbose or self.debug:
                print("Object generated successfully. 🎉")

            if self.debug:
                response_json = response.model_dump_json()
                print(f"EasyLLM Response: {response_json}")
            return response.choices[0].message.parsed
        except Exception as e:
            print(f"Failed to generate object. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def get_model(self) -> str:
        """Gets the current model."""
        return self.model

    def set_model(self, model: str) -> None:
        """Sets the model to the given model."""
        try:
            if self.verbose or self.debug:
                print(f"Setting model to {model}")
            self.openai = openai.OpenAI(api_key=self.api_key)
            self.model = model
            if self.verbose or self.debug:
                print(f"Model set to {model}")
        except Exception as e:
            print(f"Failed to set model.\nError: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def estimate_tokens(self, text: str) -> int:
        """
        Estimate the number of tokens in a given string.

        Args:
            text (str): The input string.

        Returns:
            int: Estimated token count.
        """
        # Split text by whitespace and count tokens, including punctuation
        tokens = re.findall(r"\S+", text)
        return len(tokens)

## Step 3: Define Classes, Models, and Functions for README Generation

 This step sets up classes, models, and functions to automate README generation. 

It includes defining project details, extracting badges, generating a dynamic table of contents, and configuring headers. 

The modular approach ensures flexibility, enabling well-structured and comprehensive READMEs tailored to each project.


In [ ]:
from typing import Dict, Any
import requests
import re
from pydantic import BaseModel
from typing import List


class Project(BaseModel):
    repo_name: str
    title: str
    description: str
    contributors: list[str]
    references: list[str]
    tech_stack: list[str]
    tags: list[str]
    license: str


class Sections(BaseModel):
    sections: list[str]


def extract_badges_from_markdown(markdown_text: str) -> Dict[str, str]:
    """
    Extract badges and their URLs from the given markdown text.

    Args:
        markdown_text (str): The markdown content containing badges.

    Returns:
        dict: A dictionary with badge names as keys and URLs as values.
    """
    pattern = r"\[!\[([^\]]+)\]\((https?://[^\)]+)\)\]"
    matches = re.findall(pattern, markdown_text)
    return {badge: url for badge, url in matches}


def get_badges() -> Dict[str, str]:
    """
    Get the markdown content of the README file containing badges.

    Args:
        None

    Returns:
        dict: A dictionary with badge names as keys and URLs as values.

    """
    url = (
        "https://raw.githubusercontent.com/inttter/md-badges/refs/heads/main/README.md"
    )
    response = requests.get(url)
    badges_markdown = response.text
    return extract_badges_from_markdown(badges_markdown)


def get_table_of_contents(llm: EasyLLM, project: Project) -> List[str]:
    """
    Generates the table of contents for the README.

    Args:
        llm (EasyLLM): The EasyLLM instance.
        project (Project): The Project instance.

    Returns:
        list: A list of sections for the table of contents.
    """
    mandatory_sections_first_half = [
        "About",
        "Problem Statement",
        "Research Areas",
        "Installation",
        "Usage",
    ]
    mandatory_sections_second_half = [
        "Contributing",
        "License",
        "Acknowledgements",
        "Authors",
        "References",
    ]
    table_of_contents = mandatory_sections_first_half

    prompt = f"""
        Given the table of contents for the GitHub README of Project: '{project.title}' with the following sections:
        Mandatory Sections: {json.dumps(mandatory_sections_first_half + mandatory_sections_second_half)}
        These sections should be linked to the respective sections in the README.
        These are mandatory sections for a good README.
        You can add more sections if you want as per the project requirements.
        For instance, a web scraping project would have "AI & Web Scraping" as a separate section. 
        Then, one of our projects, knowledge-grapher would have "Knowledge Graphs in Practice" and "How Google uses Knowledge Graphs at scale" as a separate section.

        Generate any 3 new sections unique, and specific to the project.
        Make sure you don't repeat the same sections and don't include the mandatory sections again.

        Project Details: '''{project.model_dump_json()}''''
    """
    sections = llm.generate_object(prompt, Sections)

    if sections and len(sections.sections) > 0:
        for section in sections.sections:
            if section not in table_of_contents:
                table_of_contents.append(section)

    for section in mandatory_sections_second_half:
        if section not in table_of_contents:
            table_of_contents.append(section)

    return table_of_contents


def get_readme_generator_config(title: str) -> Dict[str, Any]:
    README_GENERATOR_CONFIG = {
        "visual_config": {
            "header": {
                "prompt": f"""
                Generate a good header section for the README
                1. TITLE: It should contain the title of the project with an emoji. 
                Example: `# 🚀 My Awesome Project`
                Be creative with the emoji and title and try to ensure that the title is relevant to the project and README.
                TITLE: {title}

                2. BADGES: After the title add badges.
                Available BadgesL {json.dumps(get_badges())}   

                3. Message to star us on GitHub.
                Example: Please give us a ⭐ on GitHub if this project helped you!

                4. Marketing Tagline: Add a marketing tagline for the project.
                Example: #### Simplify your ETL pipelines with LitETL 🔥 — the lightweight ETL framework!
                """
            },
            "table_of_contents": {
                "prompt": """
                Generate a table of contents for the README.
                This should include "About", "Problem Statement", "Research Areas", "Installation", "Usage", "Contributing", "License", "Acknowledgements", "Authors", "References".
                These sections should be linked to the respective sections in the README.
                These are mandatory sections for a good README.
                You can add more sections if you want as per the project requirements.
                For instance, a web scraping project would have "AI & Web Scraping" as a separate section. 
                Then, one of our projects, knowledge-grapher would have "Knowledge Graphs in Practice" and "How Google uses Knowledge Graphs at scale" as a separate section.
                """
            },
        }
    }
    return README_GENERATOR_CONFIG

## Step 4: Extract and Process Badges for README

Badges are extracted from the markdown content, converted into a well-formatted JSON string, their token usage is estimated, and both the badge count and the formatted JSON representation are displayed for review.


In [ ]:
import json

llm = EasyLLM()
badges = get_badges()
badges_json = json.dumps(badges, indent=2)


print(f"Badges found in the README: {len(badges)}")
# For estimating if it will fit in the LLM context window
print(f"Tokens: {llm.estimate_tokens(badges_json)}")
print(badges_json)

 ## Step 5: Generate Table of Contents for README

 This step focuses on generating a dynamic table of contents for the README. By utilizing the EasyLLM instance and a Project object, it customizes the sections of the ToC based on both mandatory and project-specific inputs. 

The table of contents is generated, and the output is cleaned up to provide a clean and readable display before printing the final list of sections to be included in the README.

In [ ]:
from IPython.display import clear_output

table_of_contents = get_table_of_contents(
    llm,
    Project(
        repo_name="test",
        title="Test",
        description="Test",
        contributors=["Adi"],
        references=["Ref"],
        tech_stack=["Python"],
        tags=["Test"],
        license="MIT",
    ),
)

clear_output()

print(f"Table of Contents: {table_of_contents}")

## Step 6: Implementing the EasyLLM Class

The EasyLLM class provides a streamlined interface for interacting with the OpenAI API. 

It offers methods for generating text, parsing structured objects, managing model configurations, and estimating token usage. 

The class simplifies the process of text generation and provides utilities to manage prompts, handle errors, and estimate the computational load in terms of tokens, ensuring efficient use of the OpenAI model. 

By initializing with necessary configurations, it facilitates tasks like dynamic text generation and response parsing with minimal setup.


In [49]:
import os
import openai
from pydantic import BaseModel
import traceback
from typing import Union
import json
import re

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"
DEFAULT_SYSTEM_PROMPT = "You are an intelligent AI assistant. The user will give you a prompt, respond appropriately."
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 1024


class EasyLLM:
    """
    A simple abstraction for the OpenAI API. It provides easy-to-use methods to generate text and objects using the OpenAI API.
    A demonstration for the "How to build an Abstaction with Open AI API" blog post.
    Author: Aditya Patange (AdiPat)
    """

    def __init__(
        self,
        api_key=os.getenv("OPENAI_API_KEY"),
        model=DEFAULT_OPENAI_MODEL,
        verbose=True,
        debug=True,
    ):
        self.verbose = verbose
        self.debug = debug

        if self.verbose:
            print("EasyLLM: Powering up! 🚀")

        self.api_key = api_key
        self.openai = openai.OpenAI(api_key=api_key)
        self.model = model

        if self.verbose:
            print(f"EasyLLM: Model set to {model}.")
            print("EasyLLM: Ready for some Generative AI action! ⚡️")

    def generate_text(
        self,
        prompt: str,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[str, None]:
        """Generates text using the OpenAI API."""
        try:
            if self.verbose or self.debug:
                print(f"Generating text for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_tokens=max_tokens,
            )

            response = response.choices[0].message.content

            if self.verbose or self.debug:
                print("Text generated successfully. 🎉")

            if self.debug:
                response = json.dumps(response)
                print(f"EasyLLM Response: {response}")
            return response
        except Exception as e:
            print(f"Failed to generate text. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def generate_object(
        self,
        prompt: str,
        response_model: BaseModel,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[BaseModel, None]:
        """Generates an object using the OpenAI API and given response model."""
        try:
            if self.verbose or self.debug:
                print(f"Generating object for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")

            response = self.openai.beta.chat.completions.parse(
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                response_format=response_model,
                model=self.model,
                temperature=temperature,
                max_tokens=max_tokens,
            )

            if self.verbose or self.debug:
                print("Object generated successfully. 🎉")

            if self.debug:
                response_json = response.model_dump_json()
                print(f"EasyLLM Response: {response_json}")
            return response.choices[0].message.parsed
        except Exception as e:
            print(f"Failed to generate object. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def get_model(self) -> str:
        """Gets the current model."""
        return self.model

    def set_model(self, model: str) -> None:
        """Sets the model to the given model."""
        try:
            if self.verbose or self.debug:
                print(f"Setting model to {model}")
            self.openai = openai.OpenAI(api_key=self.api_key)
            self.model = model
            if self.verbose or self.debug:
                print(f"Model set to {model}")
        except Exception as e:
            print(f"Failed to set model.\nError: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def estimate_tokens(self, text: str) -> int:
        """
        Estimate the number of tokens in a given string.

        Args:
            text (str): The input string.

        Returns:
            int: Estimated token count.
        """
        # Split text by whitespace and count tokens, including punctuation
        tokens = re.findall(r"\S+", text)
        return len(tokens)

## Step 7: Generate README Configuration and Content

In Step 7, the process involves extracting badges from a markdown file, generating a table of contents, and configuring the README for a project. The Project class holds the project details, while the Sections class organizes the README sections. 

The functions `extract_badges_from_markdown` and `get_badges fetch` and `process badges`, and `get_table_of_contents` uses a custom LLM to generate a dynamic table of contents. 

The `get_readme_generator_config` function then compiles the visual configuration, including badges, header, and table of contents. The result is a detailed, structured README ready for use.


In [51]:
from typing import Dict, Any
import requests
import re
from pydantic import BaseModel
from typing import List


class Project(BaseModel):
    repo_name: str
    title: str
    description: str
    contributors: list[str]
    references: list[str]
    tech_stack: list[str]
    tags: list[str]
    license: str


class Sections(BaseModel):
    sections: list[str]


def extract_badges_from_markdown(markdown_text: str) -> Dict[str, str]:
    """
    Extract badges and their URLs from the given markdown text.

    Args:
        markdown_text (str): The markdown content containing badges.

    Returns:
        dict: A dictionary with badge names as keys and URLs as values.
    """
    pattern = r"\[!\[([^\]]+)\]\((https?://[^\)]+)\)\]"
    matches = re.findall(pattern, markdown_text)
    return {badge: url for badge, url in matches}


def get_badges() -> Dict[str, str]:
    """
    Get the markdown content of the README file containing badges.

    Args:
        None

    Returns:
        dict: A dictionary with badge names as keys and URLs as values.

    """
    url = (
        "https://raw.githubusercontent.com/inttter/md-badges/refs/heads/main/README.md"
    )
    response = requests.get(url)
    badges_markdown = response.text
    return extract_badges_from_markdown(badges_markdown)


def get_table_of_contents(llm: EasyLLM, project: Project) -> List[str]:
    """
    Generates the table of contents for the README.

    Args:
        llm (EasyLLM): The EasyLLM instance.
        project (Project): The Project instance.

    Returns:
        list: A list of sections for the table of contents.
    """
    mandatory_sections_first_half = [
        "About",
        "Problem Statement",
        "Research Areas",
        "Installation",
        "Usage",
    ]
    mandatory_sections_second_half = [
        "Contributing",
        "License",
        "Acknowledgements",
        "Authors",
        "References",
    ]
    table_of_contents = mandatory_sections_first_half

    prompt = f"""
        Given the table of contents for the GitHub README of Project: '{project.title}' with the following sections:
        Mandatory Sections: {json.dumps(mandatory_sections_first_half + mandatory_sections_second_half)}
        These sections should be linked to the respective sections in the README.
        These are mandatory sections for a good README.
        You can add more sections if you want as per the project requirements.
        For instance, a web scraping project would have "AI & Web Scraping" as a separate section. 
        Then, one of our projects, knowledge-grapher would have "Knowledge Graphs in Practice" and "How Google uses Knowledge Graphs at scale" as a separate section.

        Generate any 3 new sections unique, and specific to the project.
        Make sure you don't repeat the same sections and don't include the mandatory sections again.

        Project Details: '''{project.model_dump_json()}''''
    """
    sections = llm.generate_object(prompt, Sections)

    if sections and len(sections.sections) > 0:
        for section in sections.sections:
            if section not in table_of_contents:
                table_of_contents.append(section)

    for section in mandatory_sections_second_half:
        if section not in table_of_contents:
            table_of_contents.append(section)

    return table_of_contents


def get_readme_generator_config(title: str) -> Dict[str, Any]:
    README_GENERATOR_CONFIG = {
        "visual_config": {
            "header": {
                "prompt": f"""
                Generate a good header section for the README
                1. TITLE: It should contain the title of the project with an emoji. 
                Example: `# 🚀 My Awesome Project`
                Be creative with the emoji and title and try to ensure that the title is relevant to the project and README.
                TITLE: {title}

                2. BADGES: After the title add badges.
                Available BadgesL {json.dumps(get_badges())}   

                3. Message to star us on GitHub.
                Example: Please give us a ⭐ on GitHub if this project helped you!

                4. Marketing Tagline: Add a marketing tagline for the project.
                Example: #### Simplify your ETL pipelines with LitETL 🔥 — the lightweight ETL framework!
                """
            },
            "table_of_contents": {
                "prompt": """
                Generate a table of contents for the README.
                This should include "About", "Problem Statement", "Research Areas", "Installation", "Usage", "Contributing", "License", "Acknowledgements", "Authors", "References".
                These sections should be linked to the respective sections in the README.
                These are mandatory sections for a good README.
                You can add more sections if you want as per the project requirements.
                For instance, a web scraping project would have "AI & Web Scraping" as a separate section. 
                Then, one of our projects, knowledge-grapher would have "Knowledge Graphs in Practice" and "How Google uses Knowledge Graphs at scale" as a separate section.
                """
            },
        }
    }
    return README_GENERATOR_CONFIG

## Step 8: Retrieve and Format Badges, Estimate Token Usage

In this step, the process revolves around retrieving badges from a markdown file, formatting them into JSON, and estimating token usage.

The code initiates an EasyLLM instance to interact with the language model, retrieves badges through `get_badges()`, and converts them into a JSON string using `json.dumps()`. The number of badges is printed to provide insight into the extraction process, and token usage is estimated using the `estimate_tokens()` method to ensure the resulting data fits within the model's token limit. Finally, the JSON representation of the badges is displayed for clarity.

This workflow ensures that badge data is extracted, formatted, and optimized for processing with the language model.

In [ ]:
import json

llm = EasyLLM()
badges = get_badges()
badges_json = json.dumps(badges, indent=2)


print(f"Badges found in the README: {len(badges)}")
# For estimating if it will fit in the LLM context window
print(f"Tokens: {llm.estimate_tokens(badges_json)}")
print(badges_json)

## Step 9: Generating Table of Contents for README

In Step 9, the code focuses on generating and displaying a table of contents for a README file using a Project instance and a language model (LLM).

### Key Points

- **`clear_output()`**: Clears the output in the Jupyter Notebook to keep the interface clean and organized.

- **`get_table_of_contents()`**: This function generates the table of contents using a Project instance. The Project instance includes attributes like `repo_name`, `title`, `contributors`, and others. The `llm` instance helps suggest additional sections based on the project’s content.

- **Printing the Table of Contents**: After generating the table of contents, it is printed to show the sections that should be included in the README.

### Purpose

The objective is to dynamically generate a structured list of sections for the README, ensuring that the content is tailored to the specific project. This process enhances the organization and clarity of the notebook's output.


In [ ]:
from IPython.display import clear_output

table_of_contents = get_table_of_contents(
    llm,
    Project(
        repo_name="test",
        title="Test",
        description="Test",
        contributors=["Adi"],
        references=["Ref"],
        tech_stack=["Python"],
        tags=["Test"],
        license="MIT",
    ),
)

clear_output()

print(f"Table of Contents: {table_of_contents}")

## Conclusion:

This application streamlines the process of generating structured and informative README files for projects by automating key components such as the table of contents, badge extraction, and visual configuration. By leveraging a large language model (LLM) and predefined templates, the app ensures that each README is comprehensive and aligned with best practices.

- ### Key highlights of the app:

    - `Automated Table of Contents Generation`: It dynamically creates a table of contents based on mandatory and custom sections, tailored to each project's specific needs.
    
    - `Badge Integration`: The app can fetch badges from external markdown sources and include them in the README for a more polished presentation.

    - `Customizable Configuration`: Through configurable prompts, it generates headers, section titles, and marketing taglines, ensuring a professional and engaging README.

    - `Token Estimation for LLM`: The app estimates token usage for large data to prevent issues with token limits when working with the LLM.
    
Overall, this app significantly reduces manual effort, enhances the quality of README files, and provides a consistent and automated way to document projects effectively.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>
